### 导入相关的包

In [1]:
"""
original code from rwightman:
https://github.com/rwightman/pytorch-image-models/blob/master/timm/models/vision_transformer.py
"""

from functools import partial
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
from tqdm import tqdm
from PIL import Image

import os
import sys
import json
import pickle
import random
import math
import argparse

import matplotlib.pyplot as plt


### 定义模型板块、模型类、模型函数

In [ ]:

'''
实现随机深度（Stochastic Depth）

    在ResNet中，每一层的输出都与其输入相加，这样会导致网络的深度不断增加，导致网络的复杂度不断增加。

    而随机深度（Stochastic Depth）是一种正则化方法，通过随机丢弃网络的某些层来减少网络的深度，从而提升网络的性能。

    随机深度的实现方法是：在每一层的输出前面加一个丢弃层（DropPath），丢弃层的丢弃率是可学习的，
    这样就可以在训练过程中，根据丢弃率来随机丢弃网络的某些层，从而达到随机深度的效果。

x:输入张量
drop_prob:丢弃率，是一个浮点数，即每个路径（或神经元连接）被丢弃的概率。
training:是否在训练模式下运行。如果不是，则不进行丢弃，即丢弃率为零。
'''
def drop_path(x, drop_prob: float = 0., training: bool = False):
    """
    Drop paths (Stochastic Depth) per sample (when applied in main path of residual blocks).
    This is the same as the DropConnect impl I created for EfficientNet, etc networks, however,
    the original name is misleading as 'Drop Connect' is a different form of dropout in a separate paper...
    See discussion: https://github.com/tensorflow/tpu/issues/494#issuecomment-532968956 ... I've opted for
    changing the layer and argument names to 'drop path' rather than mix DropConnect as a layer name and use
    'survival rate' as the argument.
    """
    if drop_prob == 0. or not training:
        return x               # 如果丢弃率为0或者不进行训练，则返回x
    keep_prob = 1 - drop_prob  # 计算保留概率

    # 生成随机张量
    shape = (x.shape[0],) + (1,) * (x.ndim - 1)  # 使用diff-dim张量，而不仅仅是2D卷积网络。
    # 创建一个与输入x形状相匹配的随机张量形状,但除了第一个维度（通常是批次大小）外，其他维度都是1这样做是为了确保随机性可以广播到 x 的所有元素上
    # x.shape: 这是一个元组，表示张量 x 的形状
    # x.shape[0]: 这是x形状的第一个元素，即张量x在第一个维度上的大小
    # (1,) * (x.ndim - 1): 这里 (1,) 是一个只包含一个元素的元组，该元素是 1。通过乘以 (x.ndim - 1)，我们创建了一个新的元组，其中包含 x.ndim - 1 个 1
    # x.ndim: 张量x的维度数（即形状中元素的数量）。例如，如果x是三维的，那么x.ndim是3，所以 (1,) * (x.ndim - 1) 就是 (1, 1)
    # (x.shape[0],) + (1,) * (x.ndim - 1):这个表达式将上述两部分组合起来。
    # 首先，我们有一个包含 x 第一个维度大小的元组 (x.shape[0],)，然后是一个包含 x.ndim - 1个1的元组。
    #   通过将它们相加（使用元组的拼接操作），我们得到了一个新的形状

    random_tensor = keep_prob + torch.rand(shape, dtype=x.dtype, device=x.device)
    # 生成一个与x数据类型和设备相同的随机张量并将其值范围调整为[keep_prob, 1 + keep_prob)
    # shape:一个整数或者整数元组，指定了输出张量的形状
    # dtype:指定了输出张量的数据类型
    # device:指定了输出张量所在的设备;如果不指定，PyTorch 会根据当前设置来决定是在 CPU 上还是 GPU 上创建张量

    random_tensor.floor_()  # binarize（二值化）
    # random_tensor.floor_() 是一个原地（in-place）操作,对 random_tensor 中的每个元素进行下取整操作,即将每个元素替换为不大于该元素的最大整数

    output = x.div(keep_prob) * random_tensor
    # .div(keep_prob): 对张量 x 中的每个元素都除以 keep_prob。这是一个逐元素（element-wise）的操作。
    # random_tensor: 另一个PyTorch张量，它可能包含随机生成的数值。这个张量的形状（shape）应该与 x.div(keep_prob) 的结果相匹配，
    #   以便能够进行逐元素的乘法运算
    # 将x除以keep_prob的结果中的每个元素与random_tensor中对应位置的元素相乘。

    return output

class DropPath(nn.Module):
    """
    每个样本的路径丢弃（随机深度）（当这种技术被应用于残差块（residual blocks）的主路径上时）
    这种路径丢弃是针对每个输入样本独立进行的，即每个样本在通过网络时都可能会遇到不同的路径丢弃情况
    残差块是一种常用于深度残差网络（如ResNet）的结构，它通过一个“跳跃连接”（或称为“捷径”）将输入直接加到输出上，有助于缓解深度网络中的梯度消失问题
    """
    def __init__(self, drop_prob=None):
        super(DropPath, self).__init__()
        self.drop_prob = drop_prob
    # 第一行: 定义了DropPath类的构造函数.接受一个名为drop_prob的参数，该参数有一个默认值None
    # 第二行: 调用了DropPath类的父类的构造函数。在Python中，super()函数用于调用父类（超类）的一个方法。
    #   这里，它确保了DropPath类的实例在创建时能够正确地初始化其父类部分。
    # 第三行: self.drop_prob = drop_prob: 将传入的drop_prob（丢弃概率）保存到类的实例变量中。如果没有提供drop_prob，则默认为None

    def forward(self, x):
        return drop_path(x, self.drop_prob, self.training)
    # 定义了模块的前向传播逻辑
    # 调用drop_path函数，将输入张量x、丢弃概率self.drop_prob和当前模块的训练模式self.training作为参数传入。然后返回drop_path函数的输出。



'''
将输入的2D图像转化为一个序列的嵌入向量
定义了一个名为PatchEmbed的类，是nn.Module的子类，用于将2D图像分割成小块（patches），并将这些小块嵌入到一个更高维度的空间中
'''
class PatchEmbed(nn.Module):

    # in_c: 输入通道数，3，通常为RGB。
    # patch_size=16: 将图像分割成的小块（patches）的尺寸

    # embed_dim=768: 嵌入向量的维度，即每个图像小块被转换成的向量的长度
    # norm_layer=None: 用于嵌入向量之后的归一化层，如果为None，则不使用归一化
        
    # img_size = (img_size, img_size): 将输入的单个维度值转换为元组，表示图像的高度和宽度。
    # patch_size = (patch_size, patch_size): 同理，将patch_size也转换为元组。
        
    # self.img_size, self.patch_size: 存储处理后的图像和小块尺寸。
    # self.grid_size: 计算图像被分割成的网格大小，即小块的数量（高度和宽度方向上的小块数）。
    # self.num_patches: 图像中总的小块数量。

    # 我在想这里的img_size和patch_size直接以元组的形式传入是否会更方便。

    def __init__(self, img_size=224, patch_size=16, in_c=3, embed_dim=768, norm_layer=None):    
    # img_size: 假设图像是正方形的，所以只给出一个维度值
        super().__init__()
        img_size = (img_size, img_size)               # 将图片大小和patch大小都转换为元组，方便后面的计算
        patch_size = (patch_size, patch_size)
        self.img_size = img_size
        self.patch_size = patch_size
        self.grid_size = (img_size[0] // patch_size[0], img_size[1] // patch_size[1]) # 计算patch网格大小
        self.num_patches = self.grid_size[0] * self.grid_size[1]  # patchs的数量

        self.proj = nn.Conv2d(in_c, embed_dim, kernel_size=patch_size, stride=patch_size)
        # 使用一个二维卷积层来将每个小块从in_c个通道映射到embed_dim维的嵌入向量。卷积核的大小和步长都设置为patch_size，
        # 这样每个卷积操作就会覆盖一个小块，并且不会重叠。
                
        '''
        为何这里的卷积不需要padding呢？
        因为stride=kernel_size，每个像素块在一次卷积中都只参与一次。
        而padding适用于中间部分像素块由重复参与卷积的情况。
        '''

        self.norm = norm_layer(embed_dim) if norm_layer else nn.Identity()
        # 如果提供了norm_layer（一个归一化层的类），则创建一个该类的实例，并将其应用于嵌入向量。
        # 如果norm_layer为None，则使用nn.Identity()，它是一个不执行任何操作的占位符，相当于没有归一化层。

    # 定义用于前向传播的标准方法。该方法接收输入数据x，并返回模型的输出
    def forward(self, x):
        B, C, H, W = x.shape
        # B, C, H, W = x.shape:从输入数据x中提取其形状，其中B是批次大小（batch size），C是通道数，H是高度，W是宽度
        # 要求输入的图片大小必须和初始化时设置的img_size一致

        assert H == self.img_size[0] and W == self.img_size[1], \
            f"Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]})."
        # 使用assert语句来检查输入图像的高度和宽度是否与模型期望的尺寸相匹配。如果不匹配，将抛出一个错误消息

        # flatten: [B, C, H, W] -> [B, C, HW]
        # transpose: [B, C, HW] -> [B, HW, C]
        x = self.proj(x).flatten(2).transpose(1, 2)
        # proj(x)：用二维卷积进行投影，其实就是切割缩小啦。
        # .flatten(2): 将卷积层的输出在最后一个维度上展平
        # 使得形状从[B, embed_dim, H', W']（其中H'和W'是分割后图像的高度和宽度）变为[B, embed_dim, H'W']。

        x = self.norm(x)
        # 将归一化层（self.norm）应用于嵌入向量

        return x


'''
实现多头注意力机制

dim: 输入token的dim
num_heads: 多头注意力机制的头数
qkv_bias: 在查询Q、键K、值V矩阵的线性变换中是否添加偏置
qk_scale: 用来缩放查询Q、键K矩阵的尺度。默认是None，表示不缩放。
attn_drop_ratio: 注意力权重矩阵的dropout的概率
proj_drop_ratio: 投影矩阵的dropout的概率
'''
class Attention(nn.Module):
    def __init__(self,
                 dim,   # 输入token的dim，在Transformer模型中，这通常也是嵌入层的维度
                 num_heads=8,   # 多头注意力的头数。通过分割输入维度到多个头，模型能够并行处理多个表示子空间的信息
                 qkv_bias=False,    # 一个布尔值，指示在qkv线性变换中是否添加偏置项
                 qk_scale=None,     # 查询（query）和键（key）的缩放因子
                 attn_drop_ratio=0.,    # 注意力分数上的dropout比率
                 proj_drop_ratio=0.):   # 投影后的dropout比率。这同样用于正则化，但应用于注意力机制输出之后的投影层
        super(Attention, self).__init__()

        self.num_heads = num_heads      # 存储头数以便后续使用
        head_dim = dim // num_heads     # 每个头的维度，这里假设dim能被num_heads整除
        self.scale = qk_scale or head_dim ** -0.5       # 缩放因子，论文中默认为head_dim的倒数。
        # 如果参数中没有指定qk_scale，则使用默认的缩放因子。

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)    # 生成q、k、v三个矩阵的组合 
        # 定义一个线性层，它将输入映射到三倍于输入维度的空间
        # 生成q、k、v三个矩阵所用到的权重矩阵是Linear层自己提供的，
        # 这权重矩阵的维度是[dim, dim * 3]，在初始化不用满足什么条件（只要不为零就行），因为反正他是个可学习的参数。

        # 关于q, k, v的解释
        # Q、K、V概念来源于检索系统，其中Q为Query、K为Key、V为Value。可以简单理解为Q与K进行相似度匹配，匹配后取得的结果就是V。
        # 举个例子我们在某宝上搜索东西，输入的搜索关键词就是Q，商品对应的描述就是K，Q与K匹配成功后搜索出来的商品就是V。

        self.attn_drop = nn.Dropout(attn_drop_ratio)         # 为什么不用自己定义的DropPath呢？
        # 定义一个dropout层，用于在注意力分数上应用dropout
        # 这里在添加注意力机制（Attention Mechanism）中的丢弃层（Dropout Layer）

        self.proj = nn.Linear(dim, dim)
        # 定义一个线性投影层，它将注意力机制的输出映射回原始维度。这通常用于混合不同头的输出并调整表示。

        self.proj_drop = nn.Dropout(proj_drop_ratio) 
        # 定义另一个dropout层，用于在投影后的输出上应用dropout。

    def forward(self, x):
    # 定义了一个神经网络层的前向传播函数处理图像或者序列数据

        B, N, C = x.shape
        # x的形状为[batch_size, num_patches + 1, total_embed_dim]，
        # 其中batch_size是批次大小，num_patches + 1表示将输入图像或序列分割成的块（patches）数量加一
        # 通常是为了包含类标记或其他全局特征），total_embed_dim是每个块嵌入的总维度。

        # qkv(): -> [batch_size, num_patches + 1, 3 * total_embed_dim]
        # reshape: -> [batch_size, num_patches + 1, 3, num_heads, embed_dim_per_head]
        # permute: -> [3, batch_size, num_heads, num_patches + 1, embed_dim_per_head]
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        # qkv函数：首先，通过一个线性层（self.qkv），将输入x映射到一个更大的空间，
        # 形状变为[batch_size, num_patches + 1, 3 * total_embed_dim]。
        # 这个线性层同时生成查询（query）、键（key）和值（value）向量。然后，这些向量被重塑（reshape）和置换（permute）成
        # [3, batch_size, num_heads, num_patches + 1, embed_dim_per_head]的形状
        # 其中num_heads是注意力头的数量，embed_dim_per_head是每个头的嵌入维度。

        # [batch_size, num_heads, num_patches + 1, embed_dim_per_head]
        q, k, v = qkv[0], qkv[1], qkv[2]  # make torchscript happy (cannot use tensor as tuple)
        # 分离q, k, v：将qkv张量分割成查询（q）、键（k）和值（v）三个独立的张量。

        # transpose: -> [batch_size, num_heads, embed_dim_per_head, num_patches + 1]，
        # 即调换最后两个维度，就是做个转置，因为这样两个矩阵才能行列对应及进行相乘
        # transpose只能对两个元素进行调换，而permute可以对全局进行重新排序
        # @: multiply -> [batch_size, num_heads, num_patches + 1, num_patches + 1]
        # 当张量大于三维时，会自动选取最后两维来进行矩阵乘法。
        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)
        # 计算注意力分数: 通过查询（q）和键（k）的点积（@表示矩阵乘法），并乘以一个缩放因子（self.scale，通常是嵌入维度的平方根的倒数）得到注意力分数。
        # 然后，对这些分数应用softmax函数，使得每个块的注意力分数加起来为1。dim=-1表示最后一维。
        # 最后，通过一个dropout层（self.attn_drop）来减少过拟合

        # @: multiply -> [batch_size, num_heads, num_patches + 1, embed_dim_per_head]
        # transpose: -> [batch_size, num_patches + 1, num_heads, embed_dim_per_head]
        # reshape: -> [batch_size, num_patches + 1, total_embed_dim]
        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        # 应用注意力分数: 将注意力分数与值（v）相乘，得到加权后的值。
        # 然后，通过转置和重塑操作，将这些值恢复到原始的[batch_size, num_patches + 1, total_embed_dim]形状。

        x = self.proj(x)
        x = self.proj_drop(x)
        # 输出投影: 通过一个线性层（self.proj）对加权后的值进行投影，然后通过另一个dropout层（self.proj_drop）输出最终结果

        return x


"""
MLP as used in Vision Transformer, MLP-Mixer and related networks

这里的“MLP”代表多层感知机（Multilayer Perceptron），它是一种前馈人工神经网络模型，包含一个或多个隐藏层。
在深度学习和神经网络领域，MLP是最基本的网络结构之一
这个类继承自nn.Module，并包含了两个全连接层（线性层）、一个激活层和一个丢弃层

in_features:输入特征的维度
hidden_features:隐藏层的维度，默认等于输入特征的维度
out_features:输出特征的维度，默认等于输入特征的维度
act_layer:激活层，默认是GELU
drop:dropout的概率，默认是0
"""
class Mlp(nn.Module):
    
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()

        out_features = out_features or in_features
        # 如果out_features（输出特征的数量）已经被赋值了一个非零或非假的值，则out_features保持其原有的值；
        # 如果out_features是未定义的、零或者假（如None、False、0、空字符串等），则out_features会被设置为in_features（输入特征的数量）的值

        hidden_features = hidden_features or in_features
        # 如果hidden_features已经被明确赋值，则保持其值不变；
        # 如果hidden_features未定义、为零或为假，则它被设置为in_features的值

        self.fc1 = nn.Linear(in_features, hidden_features)
        # 创建了一个线性层（也称为全连接层），命名为self.fc1。这个层的作用是对输入数据进行线性变换。
        # in_features指定了输入数据的特征数量，即输入层的维度；hidden_features指定了输出数据的特征数量，也就是这个线性层要学习的隐藏特征的维度。
        # 在实际应用中，in_features通常等于输入数据的特征数，而hidden_features是一个需要预先设定的超参数。
        
        # 如果数据小的话，下面这两层可以不要。
        self.act = act_layer()
        # 实例化了一个激活函数层，命名为self.act。激活函数是神经网络中非线性的来源，它允许网络学习复杂的非线性关系。
        # act_layer()应该是一个返回激活函数对象的函数或类调用。
        # 常见的激活函数包括ReLU（Rectified Linear Unit，修正线性单元）、Sigmoid、Tanh等。
        # 这里没有指定具体的激活函数，所以act_layer()的具体实现决定了使用哪种激活函数。

        self.fc2 = nn.Linear(hidden_features, out_features)
        # 创建了第二个线性层，命名为self.fc2。这个层的作用是对第一个线性层（或激活函数层）的输出进行进一步的线性变换。
        # hidden_features指定了输入数据的特征数量（即第一个线性层的输出特征数量），而out_features指定了输出数据的特征数量，
        # 也就是这个线性层要学习的输出特征的维度。在实际应用中，out_features通常等于网络最终输出的维度，
        # 比如在分类任务中，它可能等于类别的数量。

        self.drop = nn.Dropout(drop)
        # 创建了一个Dropout层，命名为self.drop
        # drop参数指定了在训练过程中，每个神经元被随机丢弃（即其输出被设置为0）的概率。
        # 在训练时，Dropout层会按照指定的概率丢弃一部分神经元的输出，从而迫使网络学习到更鲁棒的特征表示。
        # 在评估（或测试）阶段，Dropout层通常不会被激活，即所有的神经元都会参与计算。

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x

    # 神经网络模型中的前向传播（forward pass）函数的一部分，用于定义数据如何通过网络流动。
    # def forward(self, x): 定义了一个名为forward的方法，它是神经网络类中用于前向传播的标准方法。self指的是类的实例本身，而x是输入到网络中的数据。
    # x = self.fc1(x): 将输入x传递给网络的第一层全连接层（fully connected layer），通常简称为fc层。
    #   self.fc1表示这个全连接层是神经网络的一个属性，之前已经被定义并初始化。
    # x = self.act(x): 接下来，数据x通过一个激活函数（activation function）。
    #   self.act是激活函数的引用，它可以是ReLU、sigmoid、tanh等任何非线性函数，用于增加网络的非线性特性，帮助网络学习复杂的模式。
    # x = self.drop(x): 然后，数据x通过一个dropout层。self.drop是dropout操作的引用，它在训练过程中随机将输入单元的一部分设置为0，以减少过拟合的风险。
    #   在测试时，通常不使用dropout或者使用一个较小的dropout率。
    # x = self.fc2(x)  数据x接着被传递给第二层全连接层self.fc2。这一层同样执行线性变换，但它可能与第一层有不同的权重和偏置。
    # x = self.drop(x)  最后，数据x再次通过一个dropout层，进一步减少过拟合的风险。
    # return x      函数返回经过两层全连接层、两次激活和两次dropout处理后的数据x。
    #   这个输出可以作为网络的最终输出（例如，用于分类任务的类别分数），或者作为另一个网络部分的输入（例如，在更深的网络中）。

'''
transformer encoder 中的基本快

dim:输入token的dim
num_heads:多头注意力机制的头数
mlp_ratio:MLP的隐藏层维度与输入token的维度的比值
qkv_bias:在查询Q、键K、值V矩阵的线性变换中是否添加偏置
qk_scale:用来缩放查询Q、键K矩阵的尺度。默认是None，表示不缩放。
drop_ratio:MLP和注意力机制后的dropout的概率
attn_drop_ratio:注意力权重矩阵的dropout的概率
drop_path_ratio:随机深度丢失的概率，用于正则化。
act_layer:激活层
norm_layer:归一化层
'''

class Block(nn.Module):
    def __init__(self,
                 dim,
                 num_heads,
                 mlp_ratio=4.,
                 qkv_bias=False,
                 qk_scale=None,
                 drop_ratio=0.,
                 attn_drop_ratio=0.,
                 drop_path_ratio=0.,
                 act_layer=nn.GELU,
                 norm_layer=nn.LayerNorm):

   
        super(Block, self).__init__()
        # 在初始化函数中，首先调用super(Block, self).__init__()来初始化父类nn.Module

        self.norm1 = norm_layer(dim)
        # self.norm1 和下文的 self.norm2: 两个规范化层，用于输入到注意力机制和MLP之前的特征规范化

        self.attn = Attention(dim, num_heads=num_heads, qkv_bias=qkv_bias, qk_scale=qk_scale,
                              attn_drop_ratio=attn_drop_ratio, proj_drop_ratio=drop_ratio)
        # NOTE: drop path for stochastic depth, we shall see if this is better than dropout here
        # 如果drop_path_ratio=0，则不使用随机深度。如果drop_path_ratio>0，则使用随机深度。 
        # self.attn: 注意力机制模块，包含多头注意力（Multi-Head Attention）和可能的dropout

        self.drop_path = DropPath(drop_path_ratio) if drop_path_ratio > 0. else nn.Identity()
        # self.drop_path: 随机深度正则化，如果drop_path_ratio大于0，则使用DropPath，否则使用nn.Identity（即不进行任何操作）

        self.norm2 = norm_layer(dim)

        mlp_hidden_dim = int(dim * mlp_ratio)
        # 计算多层感知机（MLP, Multi-Layer Perceptron）中隐藏层的维度
        # 将基础维度 dim 与比例系数 mlp_ratio 相乘，然后将结果转换为整数类型，赋值给 mlp_hidden_dim
        # 为了根据原始维度和指定的比例来动态确定隐藏层的维度，以适应不同的模型需求或限制

        self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop_ratio)
        # self.mlp: 多层感知机模块，包含两个全连接层和一个激活函数
        # mlp_ratio：这是一个比例系数，用于调整隐藏层的维度大小。通过调整这个比例系数，可以控制隐藏层的复杂度和模型的能力
        # mlp_hidden_dim：这是最终计算出的隐藏层维度大小，是一个整数

    def forward(self, x):
        x = x + self.drop_path(self.attn(self.norm1(x)))
        x = x + self.drop_path(self.mlp(self.norm2(x)))
        return x
    # 一个典型的Transformer模型中的前向传播（forward pass）过程的一部分
    # 将注意力机制（Attention）和多层感知机（MLP，Multi-Layer Perceptron）这两个核心组件集成到模型中

    # 输入：x 是输入到当前层的数据。

    # 注意力机制（Attention）：
    # self.norm1(x)：首先，对输入x应用第一个归一化层（Normalization Layer），这有助于稳定训练过程并加速收敛。
    # self.attn(...)：然后，将归一化后的数据传递给注意力机制模块。注意力机制允许模型在处理输入数据时关注到最重要的部分。
    # self.drop_path(...)：接着，应用dropout路径丢弃（Drop Path），这是一种正则化技术，用于减少模型在训练过程中的过拟合风险。
    #   它通过随机丢弃一些路径（即连接）来实现。

    # 多层感知机（MLP）：
    # self.norm2(x)：在将x与注意力机制的输出相加之后，对结果应用第二个归一化层。
    # self.mlp(...)：将归一化后的数据传递给多层感知机模块。MLP通常包含多个全连接层（也称为线性层或密集层），用于进一步处理数据。
    # self.drop_path(...)：再次应用dropout路径丢弃。

    # 残差连接（Residual Connection）：
    # 在每个模块（注意力机制和MLP）之后，通过x = x + ...的形式，将原始输入x与模块的输出相加。
    #   这种设计被称为残差连接，它有助于解决深度神经网络中的梯度消失问题，使得深层网络能够更有效地训练。

    # 输出：最后，返回经过处理的数据x，作为当前层的输出，可以传递给下一层。


"""
Args:
    img_size (int, tuple): input image size
    patch_size (int, tuple): patch size
    in_c (int): number of input channels
    num_classes (int): number of classes for classification head，分类头的输出类别数
    embed_dim (int): embedding dimension
    depth (int): depth of transformer ，即block的数量
    num_heads (int): number of attention heads，多头注意力机制中头的数量
    mlp_ratio (int): ratio of mlp hidden dim to embedding dim
    qkv_bias (bool): enable bias for qkv if True
    qk_scale (float): override default qk scale of head_dim ** -0.5 if set
    representation_size (Optional[int]): enable and set representation layer (pre-logits) to this value if set
    如果有设置，表示层的维度，用于logits层
    distilled (bool): model includes a distillation token and head as in DeiT models
    是否为蒸馏模型，即是否包含蒸馏token和头（如DeiT模型）
    drop_ratio (float): dropout rate
    attn_drop_ratio (float): attention dropout rate
    drop_path_ratio (float): stochastic depth rate
    embed_layer (nn.Module): patch embedding layer
    norm_layer: (nn.Module): normalization layer

    img_size (int, tuple): 输入图像的大小。
    patch_size (int, tuple): 每个图像块的大小。
    in_c (int): 输入图像的通道数。
    num_classes (int): 分类头输出的类别数。
    embed_dim (int): 嵌入维度。
    depth (int): Transformer的深度，即Transformer块的数量。
    num_heads (int): 多头注意力机制中头的数量。
    mlp_ratio (int): MLP（多层感知机）隐藏维度与嵌入维度的比例。
    qkv_bias (bool): 如果为True，则为qkv（查询、键、值）启用偏置。
    qk_scale (float): 如果设置，则覆盖默认的qk缩放（即head_dim的-0.5次幂）。
    representation_size (Optional[int]): 如果设置，表示用于logits层的层的维度。
    distilled (bool): 模型是否包含蒸馏token和头，如DeiT模型中的那样。
    drop_ratio (float): dropout率。
    attn_drop_ratio (float): 注意力dropout率。
    drop_path_ratio (float): 随机深度率。
    embed_layer (nn.Module): 图像块嵌入层。
    norm_layer (nn.Module): 归一化层。
"""
class VisionTransformer(nn.Module):
    def __init__(self, img_size=224, patch_size=16, in_c=3, num_classes=1000,
                 embed_dim=768, depth=12, num_heads=12, mlp_ratio=4.0, qkv_bias=True,
                 qk_scale=None, representation_size=None, distilled=False, drop_ratio=0.,
                 attn_drop_ratio=0., drop_path_ratio=0., embed_layer=PatchEmbed, norm_layer=None,
                 act_layer=None):
        
        super(VisionTransformer, self).__init__()

        self.num_classes = num_classes
        # 将传入的num_classes参数值赋给实例变量self.num_classes。num_classes通常代表分类任务中的类别数量

        self.num_features = self.embed_dim = embed_dim  # num_features for consistency with other models
        # 将传入的embed_dim参数值同时赋给self.num_features和self.embed_dim。
        # embed_dim指的是嵌入向量的维度，即每个图像小块被嵌入到的高维空间的大小。num_features是为了与其他模型保持一致而设置的别名

        self.num_tokens = 2 if distilled else 1
        # 根据distilled参数的值来决定self.num_tokens的值。如果distilled为True，则self.num_tokens被设置为2，否则为1。
        # 在ViT模型中，num_tokens通常代表要处理的token（或嵌入向量）的数量。
        # 蒸馏（distillation）是一种模型压缩技术，可能涉及使用额外的token。

        norm_layer = norm_layer or partial(nn.LayerNorm, eps=1e-6)
        # 检查是否提供了norm_layer参数。如果没有提供（即为None或False等），
        # 则使用nn.LayerNorm作为默认的正则化层，并设置其eps参数为1e-6以避免除以零的错误。partial函数用于创建具有预设参数的新函数。

        act_layer = act_layer or nn.GELU
        # 检查是否提供了激活函数act_layer。如果没有提供，则默认使用高斯误差线性单元（GELU）作为激活函数

        self.patch_embed = embed_layer(img_size=img_size, patch_size=patch_size, in_c=in_c, embed_dim=embed_dim)
        # 创建了一个嵌入层（embed_layer），它通常是一个卷积层，用于将输入图像分割成小块（patches），并将这些小块嵌入到一个高维空间中

        num_patches = self.patch_embed.num_patches
        # 嵌入后得到的patch数量，这个属性通常由embed_layer在内部计算并存储。将self.patch_embed层计算出的patch数量赋值给num_patches变量

        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim)) # 分类token
        # 创建了一个类标记（class token）参数。类标记是一个特殊的嵌入向量，用于在Transformer编码器的输入序列中表示整个图像的分类信息。
        # 它被初始化为一个形状为(1, 1, embed_dim)的全零张量，并使用nn.Parameter包装，使其成为模型的可训练参数
        
        self.dist_token = nn.Parameter(torch.zeros(1, 1, embed_dim)) if distilled else None # 蒸馏token
        # 根据distilled参数的值决定是否创建一个蒸馏标记（distillation token）参数。
        # 如果distilled为True，则蒸馏标记被初始化为一个形状为(1, 1, embed_dim)的全零张量，并使用nn.Parameter包装。
        # 如果distilled为False，则self.dist_token被设置为None。

        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches + self.num_tokens, embed_dim)) # 位置嵌入
        # 创建了一个位置嵌入（position embedding）参数。
        # 位置嵌入用于向模型提供关于每个patch（以及类标记和蒸馏标记，如果有的话）在输入序列中的位置信息
        # torch.zeros：这个函数用于创建一个全零的张量（Tensor）
        # 这里，它创建了一个形状为(1, num_patches + self.num_tokens, embed_dim)的张量，其中所有元素都初始化为0

        self.pos_drop = nn.Dropout(p=drop_ratio) # 位置drop层
        # 在PyTorch框架中定义一个位置嵌入（positional embedding）的dropout层

        dpr = [x.item() for x in torch.linspace(0, drop_path_ratio, depth)]  # stochastic depth decay rule，随机深度衰减规律
        # 在PyTorch框架中创建一个列表dpr，该列表包含了一系列根据depth（深度）线性插值得到的drop path概率值。

        # torch.linspace是PyTorch中的一个函数，用于生成一个一维张量，该张量包含从start（起始值）到end（结束值）之间均匀分布的steps（步数/元素个数）个值。
        #   在这个例子中，start=0，end=drop_path_ratio，steps=depth
        #   这意味着从0开始，到drop_path_ratio结束，生成一个包含depth个元素的张量，这些元素在0和drop_path_ratio之间均匀分布。

        # [x.item() for x in ...]：
        #   这是一个列表推导式，用于遍历上一步生成的张量中的每个元素x，
        #   并调用.item()方法将每个元素（它本身是一个标量张量）转换成一个Python标量（通常是float或int类型）。
        #   .item()方法用于从只包含一个元素的张量中提取这个元素的值，并将其作为Python数值返回。

        # dpr: 最后，这个列表推导式的结果（即一系列根据depth线性插值得到的drop path概率值）被赋值给变量dpr。
        #   创建了一个名为dpr的列表，该列表包含了从0到drop_path_ratio之间根据depth均匀分布的depth个drop path概率值。
        #   这些概率值通常用于在深度神经网络中逐层应用drop path正则化，以减少模型过拟合的风险。

        self.blocks = nn.Sequential(*[
            Block(dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, qk_scale=qk_scale,
                  drop_ratio=drop_ratio, attn_drop_ratio=attn_drop_ratio, drop_path_ratio=dpr[i],
                  norm_layer=norm_layer, act_layer=act_layer)
            for i in range(depth)
        ]) 
        # 列表推导式，它根据depth的值动态创建一个Block模块的列表
        # 对于range(depth)中的每个i，都会创建一个新的Block模块，并且每个模块的参数都是根据给定的值设置的。
        # drop_ratio=drop_ratio：可能是指MLP内部的dropout率。
        # attn_drop_ratio=attn_drop_ratio：注意力层的dropout率。
        # drop_path_ratio=dpr[i]：这是每个Block特有的参数，根据dpr列表中的值来设置，用于在路径上应用dropout正则化。
        # norm_layer=norm_layer：指定用于归一化的层类型，如LayerNorm。
        # act_layer=act_layer：指定激活函数类型，如GELU或ReLU。

        self.norm = norm_layer(embed_dim)
        # 创建了一个归一化层的实例，并将其赋值给当前类的属性self.norm。这意味着，一旦这个类的实例被创建，它就可以通过self.norm来访问和使用这个归一化层了。
        # 操作维度由embed_dim给出

        # Representation layer
        if representation_size and not distilled:
            self.has_logits = True
            self.num_features = representation_size
            self.pre_logits = nn.Sequential(OrderedDict([
                ("fc", nn.Linear(embed_dim, representation_size)),
                ("act", nn.Tanh())
            ])) # 用于在分类头之前对特征进行进一步的处理
        else:
            self.has_logits = False
            self.pre_logits = nn.Identity()
        # 定义了一个分支条件
        # 根据representation_size和distilled这两个条件来决定是否添加一个预处理逻辑（pre_logits）层，以及这个层的具体配置

        # 条件判断：if representation_size and not distilled:：这个条件判断语句检查两个条件是否同时满足：
        #        representation_size不为零（或True，在Python中非零整数被视为True），并且distilled为False。

        # 当条件满足时：
        #   self.has_logits = True：设置类的属性has_logits为True，表示模型将包含一个逻辑（logits）层之前的预处理层。
        #   self.num_features = representation_size：设置类的属性num_features为representation_size，这个值通常用于指定模型输出特征的维度。
        #   self.pre_logits = nn.Sequential(OrderedDict([...]))：创建一个序列模型self.pre_logits，该模型包含一个有序字典OrderedDict，用于按顺序添加模块。
        #   在这个例子中，OrderedDict包含两个条目：
        #       ("fc", nn.Linear(embed_dim, representation_size))：一个线性层（全连接层），将嵌入维度embed_dim映射到表示维度representation_size。
        #       ("act", nn.Tanh())：一个激活函数层，使用双曲正切函数（Tanh）进行非线性变换。

        # 当条件不满足时：
        #   self.has_logits = False：设置类的属性has_logits为False，表示模型不包含逻辑层之前的预处理层。
        #   self.pre_logits = nn.Identity()：创建一个nn.Identity模块，并将其赋值给self.pre_logits。
        #   nn.Identity是一个特殊的模块，它不会改变输入，即输出与输入相同。
        #   在这里，它用作一个占位符，表示当不需要预处理层时，pre_logits不会对输入进行任何操作。

        # Classifier head(s)
        self.head = nn.Linear(self.num_features, num_classes) if num_classes > 0 else nn.Identity()
        self.head_dist = None
        if distilled:  # 如果distilled=True，则构建蒸馏头
            self.head_dist = nn.Linear(self.embed_dim, self.num_classes) if num_classes > 0 else nn.Identity()
        # 定义一个神经网络模型中的分类器头部（heads）部分。
        # 这里涉及到一个基本的分类器头部self.head以及一个可选的蒸馏头部self.head_dist，后者仅在启用蒸馏（distilled=True）时被构建。
        # 基本分类器头部 (self.head):
        #   self.head是一个线性层（nn.Linear），它将特征（self.num_features）映射到类别数量（num_classes）上。
        #   如果num_classes大于0，意味着这是一个有监督学习任务，需要预测具体的类别，因此使用nn.Linear(self.num_features, num_classes)来定义一个线性变换，将特征空间的维度转换为类别数量。
        #   如果num_classes不大于0（通常意味着不进行分类任务），则使用nn.Identity()，这是一个不执行任何操作的层，即输入直接等于输出。

        # self.head_dist是另一个可选的线性层，用于知识蒸馏。
        # 知识蒸馏是一种模型压缩技术，通过让一个小模型（学生模型）模仿一个大模型（教师模型）的行为来训练小模型。
        # 如果启用了蒸馏（distilled=True），则根据num_classes的值决定是否构建这个蒸馏头部。与基本分类器头部不同，
        # 蒸馏头部直接从嵌入维度（self.embed_dim）映射到类别数量（self.num_classes）。
        # 这意味着蒸馏头部可能基于不同的特征表示（例如，更深层的特征）进行类别预测。
        # 同样地，如果num_classes大于0，使用nn.Linear(self.embed_dim, self.num_classes)定义蒸馏头部；否则，使用nn.Identity()。
        # 通过蒸馏技术来可能地提高模型的性能或减小模型的尺寸

        # Weight init
        nn.init.trunc_normal_(self.pos_embed, std=0.02) # 使用截断正态分布初始化位置嵌入
        if self.dist_token is not None:
            nn.init.trunc_normal_(self.dist_token, std=0.02) # 使用截断正态分布初始化蒸馏token
        # 使用了截断正态分布（truncated normal distribution）来初始化位置嵌入（pos_embed）、蒸馏标记（如果存在的话，dist_token）
        # 以及类别标记（cls_token）。此外，它还通过调用self.apply(_init_vit_weights)方法对整个模型（或模型的一部分）应用了一个自定义的权重初始化函数

        # 截断正态分布初始化（nn.init.trunc_normal_）:
        #   nn.init.trunc_normal_是PyTorch中的一个函数，用于将张量（tensor）的值按照截断正态分布进行初始化。
        #   截断正态分布是正态分布的一种变体，它在某个区间之外的值被截断（即这些值不会被采样），从而避免了极端值的出现。
        #   在这个例子中，std=0.02指定了截断正态分布的标准差。这意味着初始化的值将围绕0分布，且大部分值将落在[-0.06, 0.06]（大约是正负3倍标准差）的范围内，
        #   但由于截断的存在，实际范围可能会略有不同。
        #   self.pos_embed、self.dist_token（如果存在）、self.cls_token分别被初始化为截断正态分布。
        #   这些通常是模型中的特定参数，用于编码位置信息、蒸馏标记和类别标记。

        nn.init.trunc_normal_(self.cls_token, std=0.02) # 使用截断正态分布初始化分类token
        self.apply(_init_vit_weights) # 剩下的其他层直接用自定义的权重初始化函数来进行初始化
        # 自定义权重初始化函数（_init_vit_weights）:
        #   self.apply(_init_vit_weights)调用了apply方法，该方法将_init_vit_weights函数应用于模型的每个模块。
        #   apply函数是PyTorch中模型的一个方法，它接受一个函数作为参数，并将该函数应用于模型的每个模块（包括子模块）。
        #   _init_vit_weights很可能是一个自定义函数，用于初始化Vision Transformer（ViT）模型或类似架构的权重。
        #   这个函数可能会根据特定的规则或策略来设置权重值，比如使用不同的初始化方法、设置特定的值或范围等。


    '''
    计算模型的前向传播，但是没有包含最终的分类头。
    定义了一个深度学习模型的前向传播过程，特别是针对那些包含类别令牌（class token）、可选的蒸馏令牌（distillation token）、
    位置嵌入（position embedding）以及一系列Transformer块（blocks）的模型，如Vision Transformer（ViT）或其变体
    '''
    def forward_features(self, x):
        # [B, C, H, W] -> [B, num_patches, embed_dim]
        x = self.patch_embed(x)  # [B, 196, 768]
        # [1, 1, 768] -> [B, 1, 768]
        cls_token = self.cls_token.expand(x.shape[0], -1, -1) # 将类token扩展到batch_size维度上
        # -1表示该位置维度大小不变，具体数值代表该位置的维度大小变成这个具体数值大小。
        # shape[0]表示取x的第一维度的大小。
        if self.dist_token is None:  # 如果没有蒸馏token，则直接将cls_token和x拼接起来
            x = torch.cat((cls_token, x), dim=1)  # [B, 197, 768]
        else: # 如果有蒸馏token，则将cls_token、dist_token和x拼接起来
            x = torch.cat((cls_token, self.dist_token.expand(x.shape[0], -1, -1), x), dim=1)

        x = self.pos_drop(x + self.pos_embed) # 加入位置嵌入，然后dropout
        x = self.blocks(x)
        x = self.norm(x) 
        if self.dist_token is None: # 如果没有蒸馏token，则只取分类token的输出
            return self.pre_logits(x[:, 0])
        else: # 如果有蒸馏token，则取分类token和蒸馏token的输出
            return x[:, 0], x[:, 1]

    # forward_features 方法
    # 输入: x 是一个形状为 [B, C, H, W] 的张量，代表批次大小为 B、通道数为 C、高度为 H、宽度为 W 的输入图像。
    # Patch 嵌入: 通过 self.patch_embed(x) 将输入图像分割成小块（patches），并将每个小块嵌入到一个固定维度 embed_dim 的向量中。
    #   输出 x 的形状变为 [B, num_patches, embed_dim]，其中 num_patches 是图像被分割成的小块数量。

    # 类别令牌（Class Token）: 复制并扩展 self.cls_token，使其批次大小与输入 x 相同，但保持令牌的数量（1个）和维度不变。扩展后的形状为 [B, 1, embed_dim]。

    # 蒸馏令牌（Distillation Token, 可选）: 如果存在 self.dist_token，则同样进行复制和扩展，形状变为 [B, 1, embed_dim]。
    #   然后，将类别令牌、蒸馏令牌（如果存在）和嵌入的patches在批次维度上进行拼接。
    
    # 位置嵌入（Position Embedding）: 将位置嵌入 self.pos_embed 加到拼接后的张量上，以提供位置信息。位置嵌入通常是一个可学习的参数。
    
    # Dropout: 应用 self.pos_drop 对加了位置嵌入的张量进行dropout操作，以减少过拟合。

    # Transformer 块: 通过一系列Transformer块 self.blocks(x) 对张量进行处理，以捕捉图像中的全局信息。

    # Layer Normalization: 应用 self.norm(x) 对Transformer块的输出进行层归一化。
    
    # 输出: 如果不存在蒸馏令牌，则返回类别令牌对应的输出（经过 self.pre_logits 处理后）；如果存在蒸馏令牌，则返回类别令牌和蒸馏令牌对应的输出。



    '''
    主要的forward函数，包含了前向传播和分类头。
    '''
    def forward(self, x):
        x = self.forward_features(x)
        if self.head_dist is not None: # 如果有蒸馏头，则将分类token和蒸馏token的输出作为输入
            x, x_dist = self.head(x[0]), self.head_dist(x[1])
            if self.training and not torch.jit.is_scripting(): # 如果是在训练模式而不是在脚本模式下，则返回两个分类的结果
                # during inference, return the average of both classifier predictions
                return x, x_dist
            else: # 否则返回平均值
                return (x + x_dist) / 2
        else: # 如果没有蒸馏头，则只返回分类token的输出
            x = self.head(x)
        return x
    # forward 方法
    # 调用 forward_features: 首先调用 forward_features 方法获取特征表示。
    # 分类头（Classifier Head）: 根据是否存在蒸馏头 self.head_dist，选择不同的处理路径。
    # 如果存在蒸馏头，则分别通过 self.head 和 self.head_dist 对类别令牌和蒸馏令牌的输出进行分类。
    # 在训练模式下，直接返回两个分类器的输出；在推理模式下，返回两个分类器输出的平均值。
    # 注意：在PyTorch的脚本化（scripting）模式下，不计算平均值，而是直接返回两个分类器的输出。
    # 如果不存在蒸馏头: 直接通过 self.head 对 forward_features 的输出进行分类，并返回结果。


"""
ViT weight initialization 自定义权重初始化函数
:param m: module，模型
"""
'''
截断正态分布是一种正态分布，其概率密度函数在负无穷到正无穷之间，但是在负无穷到正无穷之间只有一小部分区域是有限的，
因此，截断正态分布就是将正态分布的概率密度函数在负无穷到正无穷之间截断，使得其概率密度函数在负无穷到正无穷之间变为一个常数，这有助于避免极端值的出现。
截断正态分布的标准差可以由参数σ来控制，σ越大，截断的区域越小，分布越集中，σ越小，截断的区域越大，分布越分散。
'''
def _init_vit_weights(m): 
    
    if isinstance(m, nn.Linear):  # 线性层中的权重初始化
        nn.init.trunc_normal_(m.weight, std=.01) # 使用截断正态分布初始化权重，标准差设置为0.01
        if m.bias is not None:
            nn.init.zeros_(m.bias)
    elif isinstance(m, nn.Conv2d): # 卷积层中的权重初始化
        nn.init.kaiming_normal_(m.weight, mode="fan_out") 
        '''
        使用Kaiming初始化权重（也称为He初始化）（因为是何凯明大神提出的），
        其考虑了前向传播和反向传播中激活值和梯度值得方差，特别适用于ReLu激活函数。
        mode="fan_out"表示根据输出单元得数量来缩放权重
        '''
        if m.bias is not None:
            nn.init.zeros_(m.bias)
    elif isinstance(m, nn.LayerNorm): # 层归一化层中的权重初始化
        nn.init.zeros_(m.bias)
        nn.init.ones_(m.weight)
# 目的是为Vision Transformer（ViT）或其相关模型中的不同层类型初始化权重

# 线性层（nn.Linear）的权重初始化：
# 使用截断正态分布（trunc_normal_）来初始化线性层的权重。截断正态分布是一种正态分布，但其值被限制在某个范围内（通常是[-2, 2]），以避免极端值。
#   这里标准差设置为0.01。
#   如果线性层有偏置项（bias），则将其初始化为0（zeros_）。

# 卷积层（nn.Conv2d）的权重初始化：
#   使用Kaiming初始化（也称为He初始化）来初始化卷积层的权重。这种初始化方法考虑了前向传播和反向传播中激活值和梯度值的方差，
#   特别适用于ReLU激活函数。mode="fan_out"表示根据输出单元的数量来缩放权重，这有助于保持整个网络中激活值和梯度的分布相对一致。
#   如果卷积层有偏置项，则同样将其初始化为0。

# 层归一化层（nn.LayerNorm）的权重初始化：
#   对于层归一化层，通常不需要对权重进行复杂的初始化，因为层归一化本身就会对输入进行归一化处理。
#   不过，如果层归一化层有权重（实际上，nn.LayerNorm在PyTorch中通常没有权重参数，只有可学习的仿射变换参数gamma和beta，
#   但这里为了通用性，我们假设它指的是这些参数），则这里将权重（如果有的话）初始化为1（ones_）。
#   对于偏置项（如果层归一化层有的话），则将其初始化为0。


'''
原论文中提到的ViT-B/16模型，是一个基础模型，其参数量为10M。
使用16×16的图像块（patch）大小，并在ImageNet-1k数据集上以224×224的输入尺寸进行预训练。
'''
def vit_base_patch16_224(num_classes: int = 1000):
    """
    ViT-Base model (ViT-B/16) from original paper (https://arxiv.org/abs/2010.11929).
    ImageNet-1k weights @ 224x224, source https://github.com/google-research/vision_transformer.
    weights ported from official Google JAX impl:
    链接: https://pan.baidu.com/s/1zqb08naP0RPqqfSXfkB2EA  密码: eu9f
    """
    model = VisionTransformer(img_size=224,
                              patch_size=16,
                              embed_dim=768,
                              depth=12,
                              num_heads=12,
                              representation_size=None,
                              num_classes=num_classes)
    return model

'''
原论文中ViT-B/16模型的ImageNet-21k版本，其参数量为11.8M。
'''
def vit_base_patch16_224_in21k(num_classes: int = 21843, has_logits: bool = True):
    """
    ViT-Base model (ViT-B/16) from original paper (https://arxiv.org/abs/2010.11929).
    ImageNet-21k weights @ 224x224, source https://github.com/google-research/vision_transformer.
    weights ported from official Google JAX impl:
    https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-vitjx/jx_vit_base_patch16_224_in21k-e5005f0a.pth
    """
    model = VisionTransformer(img_size=224,
                              patch_size=16,
                              embed_dim=768,
                              depth=12,
                              num_heads=12,
                              representation_size=768 if has_logits else None,
                              num_classes=num_classes)
    return model

'''
原论文中提到的ViT-B/32模型，是一个基础模型，其参数量为15M。
使用32×32的图像块（patch）大小，并在ImageNet-1k数据集上以224×224的输入尺寸进行预训练。
'''
def vit_base_patch32_224(num_classes: int = 1000):
    """
    ViT-Base model (ViT-B/32) from original paper (https://arxiv.org/abs/2010.11929).
    ImageNet-1k weights @ 224x224, source https://github.com/google-research/vision_transformer.
    weights ported from official Google JAX impl:
    链接: https://pan.baidu.com/s/1hCv0U8pQomwAtHBYc4hmZg  密码: s5hl
    """
    model = VisionTransformer(img_size=224,
                              patch_size=32,
                              embed_dim=768,
                              depth=12,
                              num_heads=12,
                              representation_size=None,
                              num_classes=num_classes)
    return model

'''
原论文中ViT-B/32模型的ImageNet-21k版本，其参数量为17.8M。
'''
def vit_base_patch32_224_in21k(num_classes: int = 21843, has_logits: bool = True):
    """
    ViT-Base model (ViT-B/32) from original paper (https://arxiv.org/abs/2010.11929).
    ImageNet-21k weights @ 224x224, source https://github.com/google-research/vision_transformer.
    weights ported from official Google JAX impl:
    https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-vitjx/jx_vit_base_patch32_224_in21k-8db57226.pth
    """
    model = VisionTransformer(img_size=224,
                              patch_size=32,
                              embed_dim=768,
                              depth=12,
                              num_heads=12,
                              representation_size=768 if has_logits else None,
                              num_classes=num_classes)
    return model

'''
原论文中的ViT-L/16模型，是一个大模型，其参数量为30M。
使用16×16的图像块（patch）大小，并在ImageNet-1k数据集上以224×224的输入尺寸进行预训练。
'''
def vit_large_patch16_224(num_classes: int = 1000):
    """
    ViT-Large model (ViT-L/16) from original paper (https://arxiv.org/abs/2010.11929).
    ImageNet-1k weights @ 224x224, source https://github.com/google-research/vision_transformer.
    weights ported from official Google JAX impl:
    链接: https://pan.baidu.com/s/1cxBgZJJ6qUWPSBNcE4TdRQ  密码: qqt8
    """
    model = VisionTransformer(img_size=224,
                              patch_size=16,
                              embed_dim=1024,
                              depth=24,
                              num_heads=16,
                              representation_size=None,
                              num_classes=num_classes)
    return model

'''
原论文中ViT-L/16模型的ImageNet-21k版本，其参数量为33.8M。
'''
def vit_large_patch16_224_in21k(num_classes: int = 21843, has_logits: bool = True):
    """
    ViT-Large model (ViT-L/16) from original paper (https://arxiv.org/abs/2010.11929).
    ImageNet-21k weights @ 224x224, source https://github.com/google-research/vision_transformer.
    weights ported from official Google JAX impl:
    https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-vitjx/jx_vit_large_patch16_224_in21k-606da67d.pth
    """
    model = VisionTransformer(img_size=224,
                              patch_size=16,
                              embed_dim=1024,
                              depth=24,
                              num_heads=16,
                              representation_size=1024 if has_logits else None,
                              num_classes=num_classes)
    return model

'''
原论文中提到的ViT-L/32模型，是一个大模型，其参数量为48M。
使用32×32的图像块（patch）大小，并在ImageNet-21k数据集上以224×224的输入尺寸进行预训练。
'''
def vit_large_patch32_224_in21k(num_classes: int = 21843, has_logits: bool = True):
    """
    ViT-Large model (ViT-L/32) from original paper (https://arxiv.org/abs/2010.11929).
    ImageNet-21k weights @ 224x224, source https://github.com/google-research/vision_transformer.
    weights ported from official Google JAX impl:
    https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-vitjx/jx_vit_large_patch32_224_in21k-9046d2e7.pth
    """
    model = VisionTransformer(img_size=224,
                              patch_size=32,
                              embed_dim=1024,
                              depth=24,
                              num_heads=16,
                              representation_size=1024 if has_logits else None,
                              num_classes=num_classes)
    return model

'''
原论文中提到的ViT-H/14模型，是一个超大模型，其参数量为128M。
使用14×14的图像块（patch）大小，并在ImageNet-21k数据集上以224×224的输入尺寸进行预训练。
'''
def vit_huge_patch14_224_in21k(num_classes: int = 21843, has_logits: bool = True):
    """
    ViT-Huge model (ViT-H/14) from original paper (https://arxiv.org/abs/2010.11929).
    ImageNet-21k weights @ 224x224, source https://github.com/google-research/vision_transformer.
    NOTE: converted weights not currently available, too large for github release hosting.
    """
    model = VisionTransformer(img_size=224,
                              patch_size=14,
                              embed_dim=1280,
                              depth=32,
                              num_heads=16,
                              representation_size=1280 if has_logits else None,
                              num_classes=num_classes)
    return model

### 定义数据集类、数据集加载函数、数据可视化函数等

In [3]:

class MyDataSet(Dataset): # 继承Dataset，表明它是一个数据集类型。
    """自定义数据集"""

    def __init__(self, images_path: list, images_class: list, transform=None):
        self.images_path = images_path  # 图片路径列表
        self.images_class = images_class  # 图片类别列表
        self.transform = transform  # （可选）数据预处理方法，比如缩放裁剪等
        '''
        我看了一下数据集，里面的图片大小不一，而输入网络的图片大小要求是统一为224×224的，
        所以肯定要对图片进行预处理，下面进行训练的main函数中就有进行预处理的操作
        '''

    def __len__(self):
        return len(self.images_path)

    '''
    获取第item个样本的图像和标签
    '''
    def __getitem__(self, item):
        img = Image.open(self.images_path[item])
        # RGB为彩色图片，L为灰度图片
        if img.mode != 'RGB':
            raise ValueError("image: {} isn't RGB mode.".format(self.images_path[item]))
        label = self.images_class[item]

        if self.transform is not None:
            img = self.transform(img)

        return img, label

    '''
    定义一个静态方法，将多个样本合并为一个批次，处理成适合网络输入的格式

    batch: 一个批次的样本列表，格式为[(img1, label1), (img2, label2),..., (imgN, labelN)]
    '''
    @staticmethod # 静态方法，可以直接通过类名调用
    def collate_fn(batch):
        # 官方实现的default_collate可以参考
        # https://github.com/pytorch/pytorch/blob/67b7e751e6b5931a9f45274653f4f653a4e6cdf6/torch/utils/data/_utils/collate.py
        images, labels = tuple(zip(*batch)) # 解压batch列表，解压得到图片和标签两个列表

        images = torch.stack(images, dim=0) # 将图像堆叠为一个张量
        labels = torch.as_tensor(labels) # 将标签转换为张量
        return images, labels # 返回图像张量和标签张量



'''
root: 数据集根目录
val_rate: 验证集样本占总样本的比例
'''
def read_split_data(root: str, val_rate: float = 0.2):
    random.seed(0)  # 设置随机种子保证随机结果可复现
    assert os.path.exists(root), "dataset root: {} does not exist.".format(root) # 确保数据集根目录存在，否则报错

    # 遍历文件夹，一个文件夹对应一个类别
    flower_class = [cla for cla in os.listdir(root) if os.path.isdir(os.path.join(root, cla))] 
    # 排序，保证各平台顺序一致
    flower_class.sort()
    # 生成类别名称以及对应的数字索引
    class_indices = dict((k, v) for v, k in enumerate(flower_class)) # 创建一个字典，并将名称映射到对应的数字索引
    json_str = json.dumps(dict((val, key) for key, val in class_indices.items()), indent=4) # 将字典转换为json格式字符串，并缩进为4个空格
    with open('class_indices.json', 'w') as json_file: # 保存到class_indices.json文件中
        json_file.write(json_str)

    train_images_path = []  # 存储训练集的所有图片路径
    train_images_label = []  # 存储训练集图片对应索引信息
    val_images_path = []  # 存储验证集的所有图片路径
    val_images_label = []  # 存储验证集图片对应索引信息
    every_class_num = []  # 存储每个类别的样本总数
    supported = [".jpg", ".JPG", ".png", ".PNG"]  # 支持的文件后缀类型
    # 遍历每个文件夹下的文件
    for cla in flower_class:
        cla_path = os.path.join(root, cla)
        # 遍历获取supported支持的所有文件路径
        images = [os.path.join(root, cla, i) for i in os.listdir(cla_path)
                  if os.path.splitext(i)[-1] in supported]
        # 排序，保证各平台顺序一致
        images.sort()
        # 获取该类别对应的索引，也就是该文件夹下的花图片所对应的类别。
        image_class = class_indices[cla]
        # 记录该类别的样本数量
        every_class_num.append(len(images))
        # 按比例随机采样验证样本
        val_path = random.sample(images, k=int(len(images) * val_rate))

        # 遍历该类别的所有图片，将其分为训练集和验证集
        for img_path in images:
            if img_path in val_path:  # 如果该路径在采样的验证集样本中则存入验证集
                val_images_path.append(img_path)
                val_images_label.append(image_class)
            else:  # 否则存入训练集
                train_images_path.append(img_path)
                train_images_label.append(image_class)

    print("{} images were found in the dataset.".format(sum(every_class_num)))
    print("{} images for training.".format(len(train_images_path)))
    print("{} images for validation.".format(len(val_images_path)))
    assert len(train_images_path) > 0, "number of training images must greater than 0."
    assert len(val_images_path) > 0, "number of validation images must greater than 0."

    # 绘制类别分布图（可选）（自己手动修改plot_image的值）
    plot_image = False
    if plot_image:
        # 绘制每种类别个数柱状图
        plt.bar(range(len(flower_class)), every_class_num, align='center')
        # 将横坐标0,1,2,3,4替换为相应的类别名称
        plt.xticks(range(len(flower_class)), flower_class)
        # 在柱状图上添加数值标签
        for i, v in enumerate(every_class_num):
            plt.text(x=i, y=v + 5, s=str(v), ha='center')
        # 设置x坐标
        plt.xlabel('image class')
        # 设置y坐标
        plt.ylabel('number of images')
        # 设置柱状图的标题
        plt.title('flower class distribution')
        plt.show()

    return train_images_path, train_images_label, val_images_path, val_images_label


'''
可视化数据加载器中的图像及其对应的标签，就是从数据加载器中获取一个批次的图像并将它们显示出来。

data_loader:用于加载数据的迭代器，包含批量的图像和标签。
'''
def plot_data_loader_image(data_loader):
    batch_size = data_loader.batch_size # 当前批次中的样本数量
    plot_num = min(batch_size, 4) # 最多可视化4张图片

    # 读取class_indices.json文件，获取类别名称对应的数字索引
    json_path = './class_indices.json'
    assert os.path.exists(json_path), json_path + " does not exist."
    json_file = open(json_path, 'r')
    class_indices = json.load(json_file)

    for data in data_loader:        # 迭代数据加载器，每次迭代处理一个批次的数据
        images, labels = data       # 从批次数据中分离出图像和标签
        for i in range(plot_num):
            # [C, H, W] -> [H, W, C]
            img = images[i].numpy().transpose(1, 2, 0)
            # 反Normalize操作
            img = (img * [0.229, 0.224, 0.225] + [0.485, 0.456, 0.406]) * 255
            label = labels[i].item()
            plt.subplot(1, plot_num, i+1)
            plt.xlabel(class_indices[str(label)])
            plt.xticks([])  # 去掉x轴的刻度
            plt.yticks([])  # 去掉y轴的刻度
            plt.imshow(img.astype('uint8'))
        plt.show()

        # 在循环中，对于每个要可视化的图像：
        # 将图像从PyTorch的默认格式[C, H, W]（通道、高度、宽度）转换为[H, W, C]格式，以便matplotlib可以正确显示。
        # 反归一化图像：使用预训练的模型（如ResNet）时，图像通常会被归一化到[-1, 1]或[0, 1]范围。
        #   这里假设图像被归一化到了[0, 1]范围，并使用了ImageNet数据集的均值和标准差进行反归一化，以便恢复原始的像素值范围。
        # 获取标签的整数值，并使用它从class_indices字典中查找对应的类别名称。

'''
将一个列表对象序列化并保存在一个文件中。

list_info: 要序列化的列表对象。
file_name: 保存序列化结果的文件名。
'''
def write_pickle(list_info: list, file_name: str):
    with open(file_name, 'wb') as f:
        pickle.dump(list_info, f)

# 定义了一个名为 write_pickle 的函数，它接受两个参数：list_info 和 file_name。
# list_info 应该是一个列表（list），而 file_name 是一个字符串（str），表示要保存 pickle 文件的名称（包括路径，如果需要的话）

# 函数的目的是将 list_info 列表使用 pickle 模块序列化并保存到名为 file_name 的文件中。
# 序列化是将数据结构或对象状态转换为可存储或传输的格式的过程，而 pickle 是 Python 中用于序列化和反序列化 Python 对象结构的标准库

# def write_pickle(list_info: list, file_name: str):：定义了一个名为 write_pickle 的函数，
# 它接受一个列表 list_info 和一个字符串 file_name 作为参数。
# with open(file_name, 'wb') as f:：使用 with 语句打开文件（以二进制写入模式 'wb'），这样可以确保文件在操作完成后被正确关闭。f 是文件对象的引用。
# pickle.dump(list_info, f)：将 list_info 列表序列化并写入到文件对象 f 中。
# 这样，列表的内容就被保存到了指定的文件中，以后可以通过 pickle.load() 函数读取回来。

'''
读取一个序列化的文件，并返回一个列表对象。

file_name: 保存序列化结果的文件名。
'''
def read_pickle(file_name: str) -> list:
    with open(file_name, 'rb') as f:
        info_list = pickle.load(f)
        return info_list
# 定义了一个名为 read_pickle 的函数，旨在从指定的 pickle 文件中读取数据，并将其反序列化为 Python 列表。
# 函数接收一个参数 file_name，这是一个字符串，表示要读取的 pickle 文件的名称（包含路径信息，如果文件不在当前工作目录下）



### 训练和评估函数

In [4]:
'''
在一个epoch中训练模型，并返回训练损失和预测正确的样本数。

model: 待训练的模型。
optimizer: 用于更新模型参数的优化器。
data_loader: 用于加载数据的迭代器。
device: 运行模型的设备。
epoch: 当前epoch轮数。
'''
def train_one_epoch(model, optimizer, data_loader, device, epoch):

    # print(31)
    
    model.train()  # 开启训练模式
    loss_function = torch.nn.CrossEntropyLoss() # 交叉熵损失函数，用于多分类问题
    # 初始化两个张量，用于累计损失和正确预测的样本数量
    accu_loss = torch.zeros(1).to(device)  
    accu_num = torch.zeros(1).to(device)   
    optimizer.zero_grad() # 清空之前梯度的累积，准备进行新一次的反向传播。

    # print(32)
    
    sample_num = 0
    
    data_loader = tqdm(data_loader, file=sys.stdout) # 使用tqdm来包装data_loader，以便在控制台中显示进度条

    # print(33)

    for step, data in enumerate(data_loader):

        # print(34)
        
        images, labels = data # 读取当前批次的图像和标签
        sample_num += images.shape[0]

        pred = model(images.to(device)) # 将图像转移到指定的设备上进行计算，输入到模型中，得到模型预测结果
        pred_classes = torch.max(pred, dim=1)[1] # 得到模型预测结果中概率最大的类别索引
        accu_num += torch.eq(pred_classes, labels.to(device)).sum() # 计算并累计预测正确的样本数

        loss = loss_function(pred, labels.to(device)) # 计算损失
        loss.backward() # 反向传播计算梯度
        accu_loss += loss.detach() # 累计损失

        # 更新进度条信息，显示当前训练进度
        data_loader.desc = "[train epoch {}] loss: {:.3f}, acc: {:.3f}".format(epoch,
                                                                               accu_loss.item() / (step + 1),
                                                                               # accu_loss.item() / (sample_num),
                                                                               accu_num.item() / sample_num)
        # 检查损失值是否为有限值，如果不是，则输出警告并退出程序
        if not torch.isfinite(loss):
            print('WARNING: non-finite loss, ending training ', loss)
            sys.exit(1)

        optimizer.step() # 根据计算得到的梯度更新模型参数
        optimizer.zero_grad() # 清空梯度，准备进行下一次的反向传播

    return accu_loss.item() / (step + 1), accu_num.item() / sample_num # 返回平均损失和准确率
    # return accu_loss.item() / (sample), accu_num.item() / sample_num # 返回平均损失和准确率


'''
在一个epoch中评估模型，并返回验证损失和预测正确的样本数。
和上一个方法几乎是一摸一样的，其实都可以将二者合并了，只要用一个if else判断是训练还是评估就行了。
'''
@torch.no_grad() # 装饰器，用于禁止计算梯度，节省内存
def evaluate(model, data_loader, device, epoch):
    loss_function = torch.nn.CrossEntropyLoss()

    model.eval() # 开启评估模式

    accu_num = torch.zeros(1).to(device)   # 累计预测正确的样本数
    accu_loss = torch.zeros(1).to(device)  # 累计损失

    # 使用了 tqdm 库来显示进度条，并计算了模型在给定数据加载器 data_loader 上的平均损失和准确率
    sample_num = 0
    data_loader = tqdm(data_loader, file=sys.stdout)
    for step, data in enumerate(data_loader):
        images, labels = data
        sample_num += images.shape[0]

        pred = model(images.to(device))
        pred_classes = torch.max(pred, dim=1)[1]
        accu_num += torch.eq(pred_classes, labels.to(device)).sum()

        loss = loss_function(pred, labels.to(device))
        accu_loss += loss

        data_loader.desc = "[valid epoch {}] loss: {:.3f}, acc: {:.3f}".format(epoch,
                                                                               accu_loss.item() / (step + 1),
                                                                               accu_num.item() / sample_num)

    return accu_loss.item() / (step + 1), accu_num.item() / sample_num



### 训练

In [ ]:

num_classes=5
epochs=10
batch_size=1
lr=0.001
lrf=0.01
data_path="D:\\dataset\\flower_photos" # 换成自己储存数据集的路径
model_name='model1'
weights="D:\\dataset\\weight\\jx_vit_base_patch16_224_in21k-e5005f0a.pth" # 换成自己储存初始权重集的路径
freeze_layers=True
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
# 实例化模型
model = vit_base_patch16_224_in21k(num_classes=num_classes, has_logits=False).to(device)

# 创建权重保存文件夹
# if os.path.exists("./weights") is False:
#     os.makedirs("./weights")

tb_writer = SummaryWriter() # 初始化Tensorboard记录器用于记录训练与验证的损失和准确率，便于后续可视化

# 读取数据集
train_images_path, train_images_label, val_images_path, val_images_label = read_split_data(data_path)

# 数据预处理，比如裁剪，缩放，归一化等
data_transform = {
    # 训练时
    "train": transforms.Compose([transforms.RandomResizedCrop(224),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.ToTensor(),
                                 transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])]),
    # 验证时
    "val": transforms.Compose([transforms.Resize(256),
                               transforms.CenterCrop(224),
                               transforms.ToTensor(),
                               transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])}

# 实例化训练数据集
train_dataset = MyDataSet(images_path=train_images_path,
                          images_class=train_images_label,
                          transform=data_transform["train"])

# print()
# print(train_dataset)
# print()

# 实例化验证数据集
val_dataset = MyDataSet(images_path=val_images_path,
                        images_class=val_images_label,
                        transform=data_transform["val"])

batch_size = batch_size
nw = min([os.cpu_count(), batch_size if batch_size > 1 else 0, 8])  # number of workers
print('Using {} dataloader workers every process'.format(nw))

# 实例化数据加载器，支持批处理和多线程处理
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           pin_memory=True,
                                           num_workers=nw,
                                           collate_fn=train_dataset.collate_fn)

# print(train_loader)
print("数据长度：", len(train_loader.dataset))

val_loader = torch.utils.data.DataLoader(val_dataset,
                                         batch_size=batch_size,
                                         shuffle=False,
                                         pin_memory=True,
                                         num_workers=nw,
                                         collate_fn=val_dataset.collate_fn)

# print(1)

# # 载入预训练权重
if weights != "":
    assert os.path.exists(weights), "weights file: '{}' not exist.".format(weights)
    weights_dict = torch.load(weights, map_location=device)
    # 删除不需要的权重
    del_keys = ['head.weight', 'head.bias'] if model.has_logits \
        else ['pre_logits.fc.weight', 'pre_logits.fc.bias', 'head.weight', 'head.bias']
    for k in del_keys:
        del weights_dict[k]
    print(model.load_state_dict(weights_dict, strict=False))

# 冻结部分层，以便仅更新特定层的权重
if freeze_layers:
    for name, para in model.named_parameters():
        # 除head, pre_logits外，其他权重全部冻结
        if "head" not in name and "pre_logits" not in name:
            para.requires_grad_(False)
        else:
            print("training {}".format(name))

# print(2)

#  优化器与学习率调度器，使用SGD（随机梯度下降）和cosine（余弦）学习率
pg = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.SGD(pg, lr=lr, momentum=0.9, weight_decay=5E-5)
# Scheduler https://arxiv.org/pdf/1812.01187.pdf
lf = lambda x: ((1 + math.cos(x * math.pi / epochs)) / 2) * (1 - lrf) + lrf  # cosine
scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=lf)

# print(3)

for epoch in range(epochs):
    # train
    train_loss, train_acc = train_one_epoch(model=model,
                                            optimizer=optimizer,
                                            data_loader=train_loader,
                                            device=device,
                                            epoch=epoch)

    scheduler.step() # 更新学习率

    # print(4)
    
    # validate
    val_loss, val_acc = evaluate(model=model,
                                 data_loader=val_loader,
                                 device=device,
                                 epoch=epoch)

    # 记录结果与模型保存
    tags = ["train_loss", "train_acc", "val_loss", "val_acc", "learning_rate"]
    tb_writer.add_scalar(tags[0], train_loss, epoch)
    tb_writer.add_scalar(tags[1], train_acc, epoch)
    tb_writer.add_scalar(tags[2], val_loss, epoch)
    tb_writer.add_scalar(tags[3], val_acc, epoch)
    tb_writer.add_scalar(tags[4], optimizer.param_groups[0]["lr"], epoch)

    # torch.save(model.state_dict(), "./weights/model-{}.pth".format(epoch)) 
    '''
    这里是为了把权重保存到一个文件中，这样就随时可以应用于模型进行预测，不用重新训练
    但我们每次运行代码，在训练后马上就会用同一个模型进行预测，所以没有必要保存权重
    同理，上面的创建weights文件夹也就没有必要了
    下面预测部分的重新实例化模型、重新获取权重也就没有必要了，都注释掉
    '''


### 预测并显示结果

In [ ]:
# 数据预处理模型
data_transform = transforms.Compose(
    [transforms.Resize(256),
     transforms.CenterCrop(224),
     transforms.ToTensor(),
     transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

# load image
img_path = "D:\\dataset\\predict\\OIP.jpg" # 换成自己储存待预测图片的路径
assert os.path.exists(img_path), "file: '{}' dose not exist.".format(img_path)
img = Image.open(img_path)
plt.imshow(img)
# [N, C, H, W]
img = data_transform(img) # 数据预处理
# expand batch dimension
img = torch.unsqueeze(img, dim=0)

# read class_indict
json_path = './class_indices.json'
assert os.path.exists(json_path), "file: '{}' dose not exist.".format(json_path)

with open(json_path, "r") as f:
    class_indict = json.load(f)

'''训练完即预测，所以下面这部分注释掉'''
# # create model
# model = vit_base_patch16_224_in21k(num_classes=5, has_logits=False).to(device)
# # load model weights
# model_weight_path = "./weights/model-9.pth"
# model.load_state_dict(torch.load(model_weight_path, map_location=device))

# 使用PyTorch进行深度学习模型推理的一部分，旨在预测输入图像img的类别，并展示预测结果以及各类别的概率
model.eval()
# 将模型设置为评估（或推理）模式。在评估模式下，诸如Dropout和BatchNorm等特定层会按照测试时的方式运行（例如，Dropout层在评估时不会随机丢弃神经元）

with torch.no_grad():
    # 使用torch.no_grad()上下文管理器可以禁用梯度计算，这在推理过程中是不必要的，并且可以减少内存消耗、加速计算
    # predict class
    output = torch.squeeze(model(img.to(device))).cpu()
    # model(img.to(device))：将图像img移至指定的设备（如GPU），并通过模型进行前向传播。
    # torch.squeeze(...)：去除输出张量中所有维度为1的维度。这通常用于去除批处理维度（如果输入是单个图像而非批量）。
    # .cpu()：将输出张量移至CPU，以便后续处理。

    predict = torch.softmax(output, dim=0)
    predict_cla = torch.argmax(predict).numpy()
    # torch.softmax(output, dim=0)：对输出张量应用Softmax函数，得到每个类别的概率分布。
    # torch.argmax(predict)：找到概率最高的类别的索引。
    # .numpy()：将索引转换为NumPy数组（尽管在此处可能不是必需的，因为后续操作可能直接使用PyTorch张量）。

print_res = "class: {}   prob: {:.3}".format(class_indict[str(predict_cla)],
                                             predict[predict_cla].numpy())
plt.title(print_res)
for i in range(len(predict)):
    print("class: {:10}   prob: {:.3}".format(class_indict[str(i)],
                                              predict[i].numpy()))
plt.show()

